<a href="https://colab.research.google.com/github/casperg92/CD_fancyplot/blob/main/Plot_CD_spectra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### load dependencies
import os, sys
import csv
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit

In [3]:
def plot_CD(csv_file, conversion_factor, min_x, max_x, y_min, y_max, x_min_zoom=None,x_max_zoom=None,y_min_zoom=None,y_max_zoom=None, save_path=None):
    # read csv into pandas df
    df = pd.read_csv(csv_file, header=0, index_col=0)
    df = df.dropna(axis=1, how="all")
    df = df[::-1]
    
    df = df[['20','30','40','50','60','70','80','90']]
    df = df.multiply(conversion_factor)
    
    # plot
    sns.set_theme(style="ticks")
    fig, ax = plt.subplots()
    sns.lineplot(data=df, legend = "full", ax = ax, dashes=False, palette="coolwarm")
    plt.axhline(y=0, color='k', linestyle='--')
    
    ax.set(ylabel = "MRE [deg cm2 dmol-1]", xlabel = "Wavelength [nm]")
    ax.set_xlim([min_x,max_x])
    selected_rows = df.loc[min_x+5:max_x-5]
    
    if x_min_zoom != None:
        ax2 = plt.axes([0.65, 0.638, .25, .25])
        sns.lineplot(data = df, ax = ax2, dashes=False, legend = False, palette="coolwarm")
        ax2.set_box_aspect(0.7)
        ax2.set_title('Zoom')
        ax2.set_xlim([x_min_zoom,x_max_zoom])
        ax2.set_ylim ([y_min_zoom, y_max_zoom])
        ax2.set (xlabel = None)
        ax2.set (ylabel = None)
    
    ax.set_ylim(y_min, y_max)
    
    if save_path != None:
        plt.savefig(save_path)
    plt.show()

def plot_melting(csv_file, conversion_factor, wl, y_min, y_max, measure_interval=5, save_path=None):
    # read csv into pandas df
    df = pd.read_csv(csv_file, header=0, index_col=0)
    df = df.dropna(axis=1, how="all")
    df = df[::-1]
    
    df = df.multiply(conversion_factor)
    
    # plot melting correlationat 212 nm
    df2 = df.copy()
    row_220 = df.loc[wl].transpose()
    x_data = []

    for i in range(20, 20+(measure_interval*len(row_220)), measure_interval):
        x_data.append(i)
    
    sns.set_theme(style="ticks")
    fig, ax = plt.subplots()
    sns.scatterplot(x=x_data,y = row_220, legend = False, s=60, marker="s")
    # sns.lineplot(data=row_220)
    ax.set(ylabel = f"MRE [deg cm2 dmol-1] @{wl} nm", xlabel = "Temperature [°C]")
    
    ax.set_xlim([20,90])
    ax.set_ylim([y_min, y_max])
    
    try:
        # Fit curve
        def sigmoid(x, Top, Bottom, IC50, HillSlope):
            y = Bottom + (Top-Bottom)/(1+(IC50/x)**HillSlope)
            return (y)

        p0 = [-12000, -10000, 50, 5] # this is an mandatory initial guess

        popt, pcov = curve_fit(sigmoid, x_data, row_220, p0)

        plt.plot(x_data, sigmoid(x_data, popt[0], popt[1], popt[2], popt[3]))
    except:
        popt = ['undetermined','undetermined','undetermined','undetermined']
        sns.lineplot(x=x_data, y=row_220, palette=['k'])
        
    if save_path != None:
        plt.savefig(save_path)

    plt.show()
    
    print('Top:', popt[0])
    print('Bot:', popt[1])
    print('IC50:', popt[2])
    print('HillSlope:', popt[3])